In [31]:
original_prica = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]

In [32]:
from collections import defaultdict

In [33]:
price = defaultdict(int)

In [34]:
for i,p in enumerate(original_prica):
    price[i+1] = p

In [5]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 35})

In [28]:
price[15]

0

## Get the max splitting by enumerate

In [30]:
def r(n):
    #print(n)
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1,n)]
    )

In [74]:
r(10)

30

In [34]:
r(15)

45

In [22]:
import time

In [20]:
#@get_time
def fibonacci(n):
    if n  <= 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

In [35]:
start = time.time()
print(fibonacci(4))
end = time.time()
print(end-start)

3
0.0


In [45]:
mem = defaultdict()
def fibonacci_op(n):
    if n in mem:
        return mem[n]
    else: 
        if n <= 2:
            mem[n] = 1
            return n
        else:
            result = fibonacci_op(n-1) + fibonacci_op(n-2)

            mem[n] = result
            return result

In [66]:
start = time.time()
print(fibonacci_op(3))
end = time.time()
print(end-start)

3
0.0


# Analysis: How to optimize

## A Simpler Problem

### Decorator

In [63]:
def get_time(func):
    def wrapper(*args):
        start = time.time()
        func(*args)
        end = time.time()
        print('used time : {}'.format(end-start))
    return wrapper

In [13]:
def f1(func):
    def wrapper(*args,**kwargs):
        print('Started')
        func(*args,**kwargs)
        print('Ended')
    return wrapper

In [14]:
def f():
    print('HELLO')

In [15]:
f1(f)()

Started
HELLO
Ended


In [16]:
f = f1(f)

In [17]:
f()

Started
HELLO
Ended


In [67]:
print(f.__name__)

wrapper


In [7]:
@f1 #g = f1(g)
def g(a):
    print(a)

In [8]:
g('hello')

Started
hello
Ended


In [180]:
def k(*arg,**kwargs):
    print(kwargs)

In [181]:
k(6,b=5)

{'b': 5}


In [38]:
from functools import wraps

In [36]:
def memo(f):
    memo.already_computed = {}
    @wraps(f)
    def _wrap(arg):
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        return result
    return _wrap

# We use this method to solve Cut Rod probelm¶

In [85]:
p,s = max([(5,9)]+[(3,2)],key=lambda x: x[0])
print(s)

9


In [41]:
solution = {}

In [39]:
@memo
def r(n):
    """
    Args: n is the iron length
    Return: the max revenue 
    """
    max_price, max_split = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )

    solution[n] = (n - max_split, max_split)
    
    return max_price

In [42]:
r(20)

60

In [43]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 35,
             12: 0,
             13: 0,
             14: 0,
             15: 0,
             16: 0,
             17: 0,
             18: 0,
             19: 0,
             20: 0})

In [44]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2),
 16: (14, 2),
 17: (11, 6),
 18: (17, 1),
 19: (17, 2),
 20: (17, 3)}

# How do we parse solution?¶

In [45]:
def parse_solution(n):
    left_split, right_split = solution[n]
    
    if right_split == 0: return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [46]:
r(24)

75

In [47]:
parse_solution(20)

[11, 6, 3]

# Edit Distance

In [323]:
s=('A')
print(s[-1])

A


In [324]:
from functools import lru_cache
'''使用functools模块的lur_cache装饰器，可以缓存最多 maxsize 个此函数的调用结果，
从而提高程序执行的效率，特别适合于耗时的函数。参数maxsize为最多缓存的次数，如果为None，
则无限制，设置为2n时，性能最佳；如果 typed=True（注意，在 functools32 中没有此参数），
则不同参数类型的调用将分别缓存，例如 f(3) 和 f(3.0)。
'''

'使用functools模块的lur_cache装饰器，可以缓存最多 maxsize 个此函数的调用结果，\n从而提高程序执行的效率，特别适合于耗时的函数。参数maxsize为最多缓存的次数，如果为None，\n则无限制，设置为2n时，性能最佳；如果\xa0typed=True（注意，在 functools32 中没有此参数），\n则不同参数类型的调用将分别缓存，例如 f(3) 和 f(3.0)。\n'

In [375]:
solutions = {}
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    #print('string1',string1,'string2',string2)
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    
    tail_s1 = string1[-1]
    #print('tail_s1',tail_s1)
    tail_s2 = string2[-1]
    #print('tail_s2',tail_s2)
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  #edit[i-1][j]+1
        # string 1 delete tail
        
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  ##edit[i][j-1]+1
        # string 1 add tail of string2
    ]
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '') #edit[i-1][j-1]
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    #print(candidates)
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solutions[(string1, string2)] = operation 
    
    return min_distance

In [376]:
edit_distance('ACGE','BBREE')

4

In [377]:
solutions

{('A', 'B'): 'SUB A => B',
 ('A', 'BB'): 'ADD B',
 ('A', 'BBR'): 'ADD R',
 ('A', 'BBRE'): 'ADD E',
 ('A', 'BBREE'): 'ADD E',
 ('AC', 'B'): 'DEL C',
 ('AC', 'BB'): 'SUB C => B',
 ('AC', 'BBR'): 'ADD R',
 ('AC', 'BBRE'): 'ADD E',
 ('AC', 'BBREE'): 'ADD E',
 ('ACG', 'B'): 'DEL G',
 ('ACG', 'BB'): 'DEL G',
 ('ACG', 'BBR'): 'SUB G => R',
 ('ACG', 'BBRE'): 'ADD E',
 ('ACG', 'BBREE'): 'ADD E',
 ('ACGE', 'B'): 'DEL E',
 ('ACGE', 'BB'): 'DEL E',
 ('ACGE', 'BBR'): 'DEL E',
 ('ACGE', 'BBRE'): '',
 ('ACGE', 'BBREE'): 'ADD E'}

## Todo: Parse Solution is our homework

In [408]:
parse_solution={}
def p_solution(s1,s2):
    
    if len(s1)>len(s2):
        parse_solution[(s1,s2)] = solutions[(s1,s2)]
        p_solution(s1[:-1],s2)
        return parse_solution

    elif len(s1)<len(s2):
        parse_solution[(s1,s2)] = solutions[(s1,s2)]
        p_solution(s1,s2[:-1])
        return parse_solution

    if solutions[(s1,s2)]=='':
            parse_solution[(s1,s2)] = 'No operation'
    else:
        parse_solution[(s1,s2)]=solutions[(s1,s2)]
    for i in range(1,len(s1)):

        s1=s1[:-1]
        s2=s2[:-1]
        if solutions[(s1,s2)]=='':
            parse_solution[(s1,s2)] = 'No operation'
            continue

        parse_solution[(s1,s2)] = solutions[(s1,s2)]
    return parse_solution


In [409]:
p_solution('ACGE','BBREE')

{('A', 'B'): 'SUB A => B',
 ('AC', 'BB'): 'SUB C => B',
 ('ACG', 'BBR'): 'SUB G => R',
 ('ACGE', 'BBRE'): 'No operation',
 ('ACGE', 'BBREE'): 'ADD E'}

# Problem Case 3: Pinyin Auto Correction Problem

In [1]:
chinese_dataset = 'article.txt'

In [2]:
CHINESE_CHARATERS = open(chinese_dataset,encoding = 'gb18030').read()

In [3]:
import pinyin

In [10]:
pinyin.get('你好，中国',format='strip',delimiter=' ')

'ni hao ， zhong guo'

In [4]:
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

In [5]:
CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARATERS)

In [13]:
len(CHINESE_CHARATERS_COPYS)

127101458

In [6]:
import re

In [7]:
def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [16]:
CHINESE_CHARATERS_COPYS[:100]

'ci wai zi ben zhou （ yue ri ） qi chu xiao mi shou ji deng kuan ji xing wai qi yu ji xing yi zan ting'

In [8]:
from collections import Counter, defaultdict

In [9]:
pinyin_list=tokens(CHINESE_CHARATERS_COPYS)

In [10]:
PINYIN_COUNT = Counter(pinyin_list)#p(c)

In [19]:
PINYIN_COUNT.most_common(10)

[('shi', 860634),
 ('de', 809887),
 ('yi', 682478),
 ('ji', 645276),
 ('guo', 430042),
 ('zhong', 409418),
 ('zhi', 398612),
 ('xin', 359619),
 ('li', 355441),
 ('zai', 334106)]

In [295]:
def correct(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or #known语料库里的拼音集合
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])
    #print(candidates)
    return max(candidates,key=PINYIN_COUNT.get)

In [12]:
def known(words):
    'Return the pinyin in our data'
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [13]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)
    deletes = [a+b[1:] for (a,b) in pairs if b]#挨个删除字母
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) > 1]#交换两个字母
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b]#挨个替代字母
    inserts = [a+c+b for (a,b) in pairs for c in alphabet]#挨个插入字母
    return set(deletes + transposes + replaces + inserts)

In [73]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

# Test

In [296]:
correct('ri')

{'ci', 'pi', 'yi', 'mi', 'xi', 'di', 'ru', 're', 'ni', 'qi', 'bi', 'ai', 'ji', 'ti', 'rui', 'si', 'zi', 'ri', 'li'}


'yi'

In [297]:
correct('shu')

{'shui', 'shun', 'zhu', 'shu', 'su', 'she', 'sou', 'chu', 'hu', 'shi', 'shuo', 'sha', 'shou', 'shua'}


'shi'

In [243]:
correct('xiagn')

'xian'

In [14]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, text_pinyin.split()))

In [28]:
correct_sequence_pinyin('wo xiang shagn qinng hua da xue')

'wo xiang shang qing hua da xue'

# 思考题-homework？    
#### 如何在不带空格的时候完成自动修整？--> 如何完成拼音的自动分割？   
###### 提示：使用第一节课提到的语言模型!

woyaoshangqinghua
w yaoshangqinghua
wo yaoshangqinghua
woyao shangqinghua

-> DP

In [16]:
__pinyin = set(PINYIN_COUNT)

def all_pinyin():
    for _ in __pinyin:
        yield _

In [298]:
def correct_(word):
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or #known语料库里的拼音集合
                  known(edits1(word)) or
                  [word])
    #print(candidates)
    return max(candidates,key=PINYIN_COUNT.get)

In [301]:
def split_py(text):    
    s = text
    word = correct_(s)
    while word not in all_pinyin():
        s =s[1:]
        if s[1:] in all_pinyin():
            s=s[1:]
            break
        word = correct_(s)
    if word in all_pinyin():
        if correct_(s[1:])==word:
            s=s[1:]
    
    result_list.insert(0,correct_(s))
    #print(result_list)
    n = len(text)-len(s)
    s = text[:n]  
    if n!=0:
        
        split_py(s)
    return ' '.join(result_list)

In [397]:
result_list=[]
strs = "zhesihyigecesho"
print("原拼音：%s\n"%strs)
print("拼音拆分：%s"%split_py(strs))

原拼音：zhesihyigecesho

拼音拆分：zhe shi yi ge ce shi


In [398]:
result_list=[]
strs = "woxiangshagnqinnghuadaxeu"
print("原拼音：%s\n"%strs)
print("拼音拆分：%s"%split_py(strs))

原拼音：woxiangshagnqinnghuadaxeu

拼音拆分：wo xiang shang qing hua da xue


#### 没有引入语言模型，所以只能修改单个拼写错误的单词，无法基于语义修正词组